In [17]:
import PIL
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [18]:
panelA = None
panelB = None
panelC = None
img_array = None
grayimg = None

In [19]:
def upload():
    global panelA, img_array
    f_types = [("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")] 
    path = filedialog.askopenfilename(filetypes=f_types)

    if not path:
        return

    img_array = cv2.imread(path)
    img_array = cv2.resize(img_array, (300, 300))
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

    img_display = Image.fromarray(img_array)
    img_display = ImageTk.PhotoImage(img_display)

    if panelA is not None:
        panelA.destroy()

    panelA = tk.Label(frame_images, image=img_display, borderwidth=5, relief="sunken")
    panelA.image = img_display
    panelA.pack(side=tk.LEFT, padx=20)


In [20]:
def grayscale(method=1, shades=256):
    global img_array, grayimg, panelB

    if img_array is None:
        messagebox.showerror("Error", "Silakan unggah gambar terlebih dahulu!")
        return

    img_data = np.array(img_array).astype(np.int32)
    height, width, _ = img_data.shape
    grayimg = np.zeros((height, width), dtype=np.uint8)

    for i in range(height):
        for j in range(width):
            r, g, b = img_data[i, j]
            if method == 1: # Metode 1: Rata-rata (Average)
                gray = (r + g + b) / 3  
            elif method == 2: # Metode 2: Perceptual Grayscale
                gray = int(r * 0.299 + g * 0.587 + b * 0.114)
            elif method == 3: # Metode 3: Luminosity ((Max + Min)2)
                gray = (max(r, g, b) + min(r, g, b)) // 2
            elif method == 4: # Metode 4: Komponen Tunggal (Merah)
                gray = r  
            elif method == 5: # Metode 5: Komponen Tunggal (Hijau)
                gray = g   
            elif method == 6: # Metode 6: Komponen Tunggal (Biru)
                gray = b
            elif method == 7: # Metode 7: Max Komponen
                gray = max(r, g, b)
            elif method == 8: # Metode 8: Min Komponen 
                gray = min(r, g, b)
            elif method == 9: # Metode 9: Conversion Factor (255 Shades)
                conversion_factor = 255 / (shades - 1)
                avg_value = (r + g + b) / 3
                gray = int((avg_value / conversion_factor) + 0.5) * conversion_factor
            
            grayimg[i, j] = np.clip(gray, 0, 255)

    grayimg_display = Image.fromarray(grayimg)
    grayimg_display = ImageTk.PhotoImage(grayimg_display)

    if panelB is not None:
        panelB.destroy()

    panelB = tk.Label(frame_images, image=grayimg_display, borderwidth=5, relief="sunken")
    panelB.image = grayimg_display
    panelB.pack(side=tk.LEFT, padx=20)

In [21]:
def jalankan():
    method_text = pilih_method.get()
    if method_text:
        method_number = int(method_text.split(" - ")[0])
        grayscale(method_number)

In [22]:
def histogram():
    global grayimg, panelC

    if grayimg is None or grayimg.size == 0:
        messagebox.showerror("Error", "Gambar grayscale belum dibuat! Silakan klik tombol GRAYSCALE terlebih dahulu.")
        return

    histogram, _ = np.histogram(grayimg.flatten(), bins=256, range=[0,256])

    plt.figure(figsize=(6, 3))
    plt.bar(range(256), histogram, color='gray')
    plt.title("Histogram Grayscale")
    plt.xlabel("Pixel Intensity")
    plt.ylabel("Frequency")
    plt.savefig("histogram.png", bbox_inches='tight')
    plt.close()

    hist_img = Image.open("histogram.png")
    hist_img = hist_img.resize((500, 230))
    hist_tk = ImageTk.PhotoImage(hist_img)

    if panelC is not None:
        panelC.destroy()

    panelC = tk.Label(frame_scrollable, image=hist_tk, borderwidth=5, relief="sunken")
    panelC.image = hist_tk
    panelC.pack(pady=20)

In [23]:
# === Inisialisasi Tkinter ===
root = tk.Tk()
root.title("IMAGE PROCESSING")
root.geometry("900x600")

# === Membuat Canvas dengan Scrollbar ===
canvas = tk.Canvas(root)
scroll_y = tk.Scrollbar(root, orient="vertical", command=canvas.yview)

frame_scrollable = tk.Frame(canvas)

frame_scrollable.bind(
    "<Configure>",
    lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
)

canvas_frame = canvas.create_window((0, 0), window=frame_scrollable, anchor="nw")
canvas.configure(yscrollcommand=scroll_y.set)

canvas.pack(side="left", fill="both", expand=True)
scroll_y.pack(side="right", fill="y")

# === Label Utama ===
l1 = tk.Label(frame_scrollable, text="Tekan tombol untuk menjalankan fungsi yang tertulis",
              fg="white", bg="purple", relief="groove", font=('Verdana', 15))
l1.pack(fill=tk.X, pady=10)  

# === Frame untuk Tombol ===
frame_buttons = tk.Frame(frame_scrollable)
frame_buttons.pack(pady=5)

btn_upload = tk.Button(frame_buttons, text="Upload", fg="black", bg="lavender", command=lambda: root.after(100, upload))
btn_upload.pack(side=tk.LEFT, padx=10)

tk.Label(frame_buttons, text="Pilih Method:", font=("Times New Roman", 10)).pack(side=tk.LEFT, padx=5)

n = tk.StringVar()
pilih_method = ttk.Combobox(frame_buttons, width=40, textvariable=n)
pilih_method['values'] = (
    '1 - GRAYSCALE-Rata-rata',
    '2 - GRAYSCALE-Perceptual Grayscale',
    '3 - Luminosity',
    '4 - GRAYSCALE-Komponen Tunggal (Merah)',
    '5 - GRAYSCALE-Komponen Tunggal (Hijau)',
    '6 - GRAYSCALE-Komponen Tunggal (Biru)',
    '7 - GRAYSCALE-Max Komponen',
    '8 - GRAYSCALE-Min Komponen',
    '9 - GRAYSCALE-Conversion Factor (255 Shades)'
)
pilih_method.pack(side=tk.LEFT, padx=5)

btn_jalankan = tk.Button(frame_buttons, text="Jalankan", fg="black", bg="skyblue", command=lambda: root.after(100, jalankan))
btn_jalankan.pack(side=tk.LEFT, padx=10)

btn_histogram = tk.Button(frame_buttons, text="Histogram", fg="black", bg="lightblue", command=lambda: root.after(100, histogram))
btn_histogram.pack(side=tk.LEFT, padx=10)

# === Frame untuk Gambar ===
frame_images = tk.Frame(frame_scrollable)
frame_images.pack(pady=20)

In [24]:
# === Menjalankan Aplikasi ===
root.mainloop()